## Setup

some kinds of time series:
- trend(upward forexample)
- seasonality ( peaks szczyty , troughs doły)
- both of them
- white noise


Since neural networks rely on stochasticity (i.e. randomness) to initialize their parameters and gradient descent selects random batches of training data at each iteration, is perfectly normal if the outputs you see when you run the Colabs are slightly different from those shown in the video.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def plot_series(time, series, format="-", start=0, end=None, label=None):
    plt.plot(time[start:end], series[start:end], format, label=label)
    plt.xlabel("Time")
    plt.ylabel("Value")
    if label:
        plt.legend(fontsize=14)
    plt.grid(True)

## Trend and Seasonality

In [ ]:
def trend(time, slope=0):
    return slope * time

Let's create a time series that just trends upward:

In [ ]:
time = np.arange(4 * 365 + 1)
baseline = 10
series = baseline + trend(time, 0.1)

plt.figure(figsize=(10, 6))
plot_series(time, series)
plt.show()

In [ ]:
time

In [ ]:
series

Now let's generate a time series with a seasonal pattern:

In [ ]:
def seasonal_pattern(season_time):
    """Just an arbitrary pattern, you can change it if you wish"""
    return np.where(season_time < 0.4,
                    np.cos(season_time * 2 * np.pi),
                    1 / np.exp(3 * season_time))

def seasonality(time, period, amplitude=1, phase=0):
    """Repeats the same pattern at each period"""
    season_time = ((time + phase) % period) / period
    return amplitude * seasonal_pattern(season_time)

In [ ]:
amplitude = 40
series = seasonality(time, period=365, amplitude=amplitude)

plt.figure(figsize=(10, 6))
plot_series(time, series)
plt.show()

Now let's create a time series with both trend and seasonality:

In [ ]:
slope = 0.05
series = baseline + trend(time, slope) + seasonality(time, period=365, amplitude=amplitude)

plt.figure(figsize=(10, 6))
plot_series(time, series)
plt.show()

## Noise

In practice few real-life time series have such a smooth signal. They usually have some noise, and the signal-to-noise ratio can sometimes be very low. Let's generate some white noise:

In [ ]:
def white_noise(time, noise_level=1, seed=None):
    rnd = np.random.RandomState(seed)
    return rnd.randn(len(time)) * noise_level

In [ ]:
noise_level = 5
noise = white_noise(time, noise_level, seed=42)

plt.figure(figsize=(10, 6))
plot_series(time, noise)
plt.show()

Now let's add this white noise to the time series:

In [ ]:
series += noise


plt.figure(figsize=(10, 6))
plot_series(time, series)
plt.show()


# Fixed Partitioning:


- 3 parts - train validation test 
- discover seasonality - years second weeks



https://learn.udacity.com/courses/ud187/lessons/19212c74-9f30-459c-aa74-4d10abdcff23/concepts/58c32cf3-6778-4393-a2ed-a15507206fc8 

# Roll-Forward Partitioning



# Naive_Forecasting

https://learn.udacity.com/courses/ud187/lessons/19212c74-9f30-459c-aa74-4d10abdcff23/concepts/bf69efd4-ca86-4691-b42e-a60277a81a96

https://colab.research.google.com/github/tensorflow/examples/blob/master/courses/udacity_intro_to_tensorflow_for_deep_learning/l08c02_naive_forecasting.ipynb



# metrics



errors = forecast - actual

mse = np.square(errors).mean()

rmse = pierwiastek z mse

mae = np.abs(errors).mean()

mape = np.abs(errors / x_valid).mean() ( mean ratio between the absolute error and absolute value)



### measuring 

    keras.metrics.mean_absolute_error(x_valid, naive_forecst).numpy()


# moving AVERAGE 


mean of the past N values

nice;y eliminate a lofof noise

not anticipate seasonality and trend



# differencing



study the difference between the value at time t and the value one year earlier = time t - 365

we get this differenced time series which has no trend and no seasonality

than we can apply moving average

To the final forecast we need to add back the value at time t minus 365




- https://learn.udacity.com/courses/ud187/lessons/19212c74-9f30-459c-aa74-4d10abdcff23/concepts/9b131231-1b05-4e11-8ad0-1992a7c01ae6




but the forecast is still noisy 
we need to improve that by also removing the past noise using the moving average



- optimal MAE +- 4.0




time window  = for example week moth or somethin

forecast - prognoza

In [2]:
import tensorflow as tf
import keras
import numpy


dataset = tf.data.Dataset.range(10)
dataset = dataset.window(5,shift=1, drop_remainder=True)


for window_dataset in dataset:

    for val in window_dataset:
        print(val.numpy(), end = ' ')

    print()

0 1 2 3 4 
1 2 3 4 5 
2 3 4 5 6 
3 4 5 6 7 
4 5 6 7 8 
5 6 7 8 9 


2022-12-15 01:35:32.851468: W tensorflow/core/framework/dataset.cc:769] Input of Window will not be optimized because the dataset does not implement the AsGraphDefInternal() method needed to apply optimizations.
2022-12-15 01:35:32.868974: W tensorflow/core/data/root_dataset.cc:266] Optimization loop failed: CANCELLED: Operation was cancelled
2022-12-15 01:35:32.872792: W tensorflow/core/data/root_dataset.cc:266] Optimization loop failed: CANCELLED: Operation was cancelled
2022-12-15 01:35:32.876288: W tensorflow/core/data/root_dataset.cc:266] Optimization loop failed: CANCELLED: Operation was cancelled
2022-12-15 01:35:32.879303: W tensorflow/core/data/root_dataset.cc:266] Optimization loop failed: CANCELLED: Operation was cancelled
2022-12-15 01:35:32.882191: W tensorflow/core/data/root_dataset.cc:266] Optimization loop failed: CANCELLED: Operation was cancelled


In [4]:


import tensorflow as tf
import keras
import numpy


dataset = tf.data.Dataset.range(10)
dataset = dataset.window(5,shift=1, drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(5))


for window in dataset:

    print(window.numpy())

[0 1 2 3 4]
[1 2 3 4 5]
[2 3 4 5 6]
[3 4 5 6 7]
[4 5 6 7 8]
[5 6 7 8 9]


2022-12-15 01:35:46.995100: W tensorflow/core/data/root_dataset.cc:266] Optimization loop failed: CANCELLED: Operation was cancelled


In [14]:
import tensorflow as tf
import keras
import numpy


dataset = tf.data.Dataset.range(10)
dataset = dataset.window(5,shift=1, drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(5))
dataset = dataset.map(lambda window:(window:[:-1],window[-1:]))

for x,y in dataset:

    print(x.numpy(), y.numpy())

SyntaxError: invalid syntax (3382988835.py, line 9)

In [11]:
import tensorflow as tf
import keras
import numpy


dataset = tf.data.Dataset.range(10)
dataset = dataset.window(5,shift=1, drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(5))
dataset = dataset.map(lambda window:(window[:-1], window[-1:]))

for x,y in dataset:

    print(x.numpy(), y.numpy())

[0 1 2 3] [4]
[1 2 3 4] [5]
[2 3 4 5] [6]
[3 4 5 6] [7]
[4 5 6 7] [8]
[5 6 7 8] [9]


shuffling

In [10]:
import tensorflow as tf
import keras
import numpy


dataset = tf.data.Dataset.range(10)
dataset = dataset.window(5,shift=1, drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(5))
dataset = dataset.map(lambda window:(window[:-1], window[-1:]))
dataset = dataset.shuffle(buffer_size=10)


for x,y in dataset:

    print(x.numpy(), y.numpy())

[2 3 4 5] [6]
[4 5 6 7] [8]
[1 2 3 4] [5]
[0 1 2 3] [4]
[3 4 5 6] [7]
[5 6 7 8] [9]


In [13]:
import tensorflow as tf
import keras
import numpy


dataset = tf.data.Dataset.range(10)
dataset = dataset.window(5,shift=1, drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(5))
dataset = dataset.map(lambda window:(window[:-1], window[-1:]))
dataset = dataset.shuffle(buffer_size=10)
dataset = dataset.batch(2).prefetch(1)


for x,y in dataset:

    print('x =' , x.numpy(), x.numpy())
    print('y =' , x.numpy(), y.numpy())


x = [[1 2 3 4]
 [5 6 7 8]] [[1 2 3 4]
 [5 6 7 8]]
y = [[1 2 3 4]
 [5 6 7 8]] [[5]
 [9]]
x = [[2 3 4 5]
 [3 4 5 6]] [[2 3 4 5]
 [3 4 5 6]]
y = [[2 3 4 5]
 [3 4 5 6]] [[6]
 [7]]
x = [[0 1 2 3]
 [4 5 6 7]] [[0 1 2 3]
 [4 5 6 7]]
y = [[0 1 2 3]
 [4 5 6 7]] [[4]
 [8]]


In [ ]:
def window_dataset(series , window_size, batch_size=32):
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size + 1,shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size + 1))
    ds = ds.map(lambda w:(w[:-1], [-1]))
    ds = ds.shuffle(len(series))
    ds = ds.batch(batch_size).prefetch(1)

# convet time series into a data set that we can use to train the machine learning model


https://colab.research.google.com/github/tensorflow/examples/blob/master/courses/udacity_intro_to_tensorflow_for_deep_learning/l08c04_time_windows.ipynb

huber losses = quadratic for small errors just like the mean squared loss
            linear for large errors just like the mean absolute error
            

# forecasting with ML models

In progress

# RNNs


in progress